In [1]:
from skimage.io import imread, imshow, imsave  # Read and display images
from skimage.transform import rescale, resize  # Rescale function for images
import os  # Directory scan for files
import numpy as np  # Create blank matrixes for multi-channel images
from cellpose import models, io
import pyclesperanto_prototype as cle

In [2]:
# Static variables
downscale = 0.4
channels = [0,0]
model_cp = models.Cellpose(gpu=False, model_type='cyto')
#model_cp = models.CellposeModel(model_type='CP')  # ALternative cellpose model
diameter = 30
save_name_prefix = 'LABEL_IMAGE_'
img_name_prefx = 'N2V_'

# Get .sld files as list from current directory
image_lst = [file for file in os.listdir('.') if file.endswith(".tif") and file.startswith(img_name_prefx) ]  

for i, filename_img in enumerate(image_lst):
    # A For loop will start here to go through all the images in the folder
    original_img = imread(filename_img) # Reads one image for testing purposes

    # Store image dimensions in variables [8, 2, 1200, 1200] for [slices, channels, height, width]
    stack_dim = original_img.shape[0]
    height_dim = original_img.shape[-2]  # gets second to last item from list
    width_dim = original_img.shape[-1]  # gets last item from list

    height_dim_scaled = int(np.ceil(height_dim * downscale))
    width_dim_scaled = int(np.ceil(width_dim * downscale))

    # Blank matrixes
    scaled_img = np.zeros((stack_dim, height_dim_scaled, width_dim_scaled), dtype = 'float32')
    scaled_mask = np.zeros((stack_dim, height_dim_scaled, width_dim_scaled), dtype = 'float32')

    # Loop to rescale original image and store it as a matrix
    # Uses scikit rescale to downsample image so the structures of interest end with 30 px diam
    # Rescaled image is in normalized (0-1) float64, is multiplied by 255 to allow retyping to unint8
    # Once back at uint8 it is stored in a matrix
    for stack_pos in range(stack_dim):
        scaled_1 = rescale(original_img[stack_pos, :, :], downscale, anti_aliasing = True)
        # scaled_2 = scaled_1 * 255
        # scaled_3 = scaled_2.astype('uint8')
        scaled_img[stack_pos, :, :] = scaled_1
        
        #cle.imshow(scaled_img[stack_pos,:,:])

    # show = imshow(scaled_img[0, :, :])
    
    
    # Print to track analysis progress
    print(f"Currently creating labels for image {filename_img}.")
    
    # Loop to test several diameter for cellpose model 2 by 2 from 20 to 34
    for stack_pos in range(stack_dim):
        # Print to track analysis progress
        # print( str(stack_pos + 1).zfill(3) + ' of ' + str(stack_dim).zfill(3) + " of image " + filename_img)
        print('.', end='')

        # run cellpose model
        label_image, flows, styles, diams = model_cp.eval(scaled_img[stack_pos, :, :], diameter = diameter, channels = channels)
        #label_image, flows, styles = model_cp.eval(scaled_img[stack_pos, :, :], diameter = diameter, channels = channels) # Alternative cellpose model

        # store mask result in multychannel matrix for saving
        scaled_mask[stack_pos, :, :] = label_image
        
        # cle.imshow(scaled_mask[stack_pos,:,:], labels=True)

    # save both images
    imsave(save_name_prefix + '_CP_' + filename_img, scaled_mask, imagej = True)
    # imsave('scaled_img.tif', scaled_img, imagej = True)
    print('.')
    # print('-----')
    

Currently creating labels for image N2V_MAX_liveimaging - CNTB_CK_a.tif.
..........................................................................
Currently creating labels for image N2V_MAX_liveimaging - CNTB_CK_b.tif.
..........................................................................
Currently creating labels for image N2V_MAX_liveimaging - CNTB_CK_c.tif.
..........................................................................
Currently creating labels for image N2V_MAX_liveimaging - CNTB_CK_d.tif.
..........................................................................
Currently creating labels for image N2V_MAX_liveimaging - CNTB_CK_e.tif.
..........................................................................
Currently creating labels for image N2V_MAX_liveimaging - CNTB_DMSO_a.tif.
..........................................................................
Currently creating labels for image N2V_MAX_liveimaging - CNTB_DMSO_b.tif.
...................................

...

..

...

....

............

..........................

.......

C:\Users\marianaferreira\AppData\Local\Temp\ipykernel_3644\1116437157.py:63: UserWarning: LABEL_IMAGE__CP_N2V_MAX_liveimaging - DMSO_Y_a.tif is a low contrast image
  imsave(save_name_prefix + '_CP_' + filename_img, scaled_mask, imagej = True)


.
Currently creating labels for image N2V_MAX_liveimaging - DMSO_Y_b.tif.
..........................................................................
Currently creating labels for image N2V_MAX_liveimaging - DMSO_Y_c.tif.
..........................................................................
Currently creating labels for image N2V_MAX_liveimaging - DMSO_Y_d.tif.
..........................................................................
Currently creating labels for image N2V_MAX_liveimaging - DMSO_Y_e.tif.
..........................................................................


In [3]:
print('OK?')

OK?
